<a href="https://colab.research.google.com/github/Menodore/Deepfake-Detection/blob/main/deepfake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch torchvision numpy opencv-python dlib

In [ ]:

import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import numpy as np
import cv2
from torchvision.models import vgg19, inception_v3
from torch.utils.data import DataLoader, Dataset


In [ ]:
#data augumentation
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(224),
    transforms.ColorJitter(),
    transforms.ToTensor()
])


In [ ]:
# Encoder model(VGG19)
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.model = vgg19(pretrained=True).features
        self.pool = nn.AdaptiveAvgPool2d((1, 1))

    def forward(self, x):
        x = self.model(x)
        x = self.pool(x)
        return x.view(x.size(0), -1)  #flatten features

encoder = Encoder()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:06<00:00, 83.7MB/s]


In [ ]:
# Contrastive loss function
class ContrastiveLoss(nn.Module):
    def __init__(self, temperature=0.5):
        super(ContrastiveLoss, self).__init__()
        self.temperature = temperature

    def forward(self, z_i, z_j):
        #Normalize features
        z_i = nn.functional.normalize(z_i, dim=1)
        z_j = nn.functional.normalize(z_j, dim=1)

        #pairwise cosine similarities
        similarities = torch.mm(z_i, z_j.T) / self.temperature

        #Contrastive loss
        loss = -torch.mean(torch.log(torch.exp(similarities.diag()) / torch.sum(torch.exp(similarities), dim=1)))
        return loss

contrastive_loss = ContrastiveLoss()


In [ ]:
# CNN Classifier
class Classifier(nn.Module):
    def __init__(self, input_dim=512, num_classes=2):
        super(Classifier, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 2048),
            nn.ReLU(),
            nn.Linear(2048, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.fc(x)

classifier = Classifier(input_dim=512, num_classes=2)


In [ ]:
#training loop for contrastive learning
def train_contrastive(encoder, loader, optimizer, loss_fn, epochs=10):
    encoder.train()
    for epoch in range(epochs):
        for i, (img1, img2) in enumerate(loader):
            img1, img2 = img1.cuda(), img2.cuda()
            z1, z2 = encoder(img1), encoder(img2)
            loss = loss_fn(z1, z2)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if i % 10 == 0:
                print(f"Epoch [{epoch}/{epochs}], Step [{i}], Loss: {loss.item():.4f}")


In [ ]:
from PIL import Image

def preprocess_image(image_path, transform):
    """

    Args:
      image_path:
      transform:

    Returns:

    """
    # Load image
    image = Image.open(image_path).convert('RGB')  # Convert to RGB
    # Apply transformations (resize, normalize, etc.)
    image = transform(image)
    # Add a batch dimension
    return image.unsqueeze(0)  # Shape: [1, C, H, W]


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match model input size
    transforms.ToTensor(),         # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])


In [ ]:
# Load and preprocess the image
image_path = "/content/sample_data/WIN_20241204_14_46_54_Pro.jpg"
image_tensor = preprocess_image(image_path, transform)

# Move image and model to the same device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
image_tensor = image_tensor.to(device)
encoder = encoder.to(device)

# feature vector
encoder.eval()  # setting model to evaluation mode
with torch.no_grad():
    feature_vector = encoder(image_tensor)  # Output the feature vector
print("Feature Vector Shape:", feature_vector.shape)


Feature Vector Shape: torch.Size([1, 512])


In [ ]:
# setting classifier on device
classifier = classifier.to(device)

# preditcion
classifier.eval()
with torch.no_grad():
    prediction = classifier(feature_vector)
    predicted_class = torch.argmax(prediction, dim=1).item()

label = {0: "Real", 1: "Fake"}
print("Prediction:", label[predicted_class])


Prediction: Fake


In [ ]:


#video file
video_path = "/content/WIN_20241204_14_46_59_Pro.mp4"

#extract frames from a video
def extract_frames(video_path, frame_rate=1):
    frames = []
    cap = cv2.VideoCapture(video_path)
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Capture a frame at the specified frame rate
        if count % frame_rate == 0:
            frames.append(frame)
        count += 1
    cap.release()
    return frames

# Extract frames from the video
frames = extract_frames(video_path, frame_rate=30)  # Adjust frame_rate as needed

# Preprocess each frame
processed_frames = [preprocess_image(frame, transform) for frame in frames]

# Move models to the same device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
encoder = encoder.to(device)
classifier = classifier.to(device)

# Classify each frame
frame_predictions = []
encoder.eval()
classifier.eval()

with torch.no_grad():
    for frame in processed_frames:
        # Move frame to device
        frame = frame.to(device)
        # Get feature vector
        feature_vector = encoder(frame)
        # Classify frame
        prediction = classifier(feature_vector)
        frame_predictions.append(torch.argmax(prediction, dim=1).item())

# Majority Voting
final_prediction = max(set(frame_predictions), key=frame_predictions.count)

# Map prediction to labels
labels = {0: "Real", 1: "Fake"}
print(f"The video is predicted to be: {labels[final_prediction]}")


AttributeError: 'numpy.ndarray' object has no attribute 'read'